## Restaurant Seat Optimization




#### Step 0: Obtain parameters from 'data.txt'



In [1]:
import json

w = l = customer_goodwill = REVENUE_GOAL = num_timeslots = randomly_generated_data = max_party_size = 0
marginal_revenue = {}

with open('../data.json') as f:
    data = json.load(f)
    w = data['restaurant_width']
    l = data['restaurant_length']
    customer_goodwill = data['customer_goodwill']
    REVENUE_GOAL = data['revenue_goal']
    randomly_generated_data = data['randomly_generated_data']
    max_party_size = data['max_party_size']
    for i in data['marginal_revenue']:
        marginal_revenue[int(i)] = data['marginal_revenue'][i]

#### Step 1: Import gurobipy module

In [2]:
import gurobipy as gp
from gurobipy import GRB
import random

#### Step 2: Define your model

In [3]:
m = gp.Model()

Using license file /Users/matthewfollegot/gurobi.lic
Academic license - for non-commercial use only


#### Step 3: Define your sets

In [4]:
# set of timeslots
K = 7

# set of parties at each timeslot
# since there are 7 timeslots and since this data is randomly generated, 
# the set of parties during timelsot k will be I[k].
# Prior to having randomly generated demand we had set I = 40 (40 parties) for each timeslot
if randomly_generated_data == True:
    I = [
        random.randint(10,15), 
        random.randint(17,24),
        random.randint(17,24),
        random.randint(10,15),
        random.randint(25,40),
        random.randint(25,40),
        random.randint(10,15),
    ]

else:
    print("Please allow data to be randomly generated to ensure accuracy and reliability of resutls")

#### Step 4: Define your parameters

In [5]:
#initializing the party 
party_size = {}
for k in range(K):
    for i in range(I[k]):
        # TODO: make this an input var from the user and make the table_mapping dynamic based on this
        party_size[i, k] = random.randint(2,max_party_size)

# the table mapping below represents
# how many tables a specific party_size would require
table_mapping = {}
for i in range(2,max_party_size + 1):
    table_mapping[i] = i // 2 + i % 2
print(table_mapping)

# initializing the amount of space that each of the party sizes 
# would take up
space = {}
for k in range(K):
    for i in range(I[k]):
        num_tables = table_mapping.get(party_size[i, k])
        space[i, k] = 6 + 3 * num_tables

{2: 1, 3: 2, 4: 2, 5: 3, 6: 3}


#### Step 5: Define your decision variables

In [6]:
# defining xik, whethere or not party i is seated at timeslot k
x = {}
for k in range(K):
    for i in range(I[k]):
        x[i,k] = m.addVar(lb=0.0, vtype=GRB.BINARY, name="x_"+str(i)+str(k))
        
#defining b_k, whether or not there is a social distancing violation during timeslot k
b={}
for k in range(K):
    b[k] = m.addVar(lb=0.0, obj=880, vtype=GRB.BINARY, name="b_"+str(k))
    
s = {}
#defining space slack variable
for k in range(K):
    s[k] = m.addVar(lb=0.0, vtype=GRB.INTEGER, name="s_"+str(k))

#defining space excess variable
e = {}
for k in range(K):
    e[k] = m.addVar(lb=0.0, vtype=GRB.INTEGER, name="e_" + str(k))

# defining revenue goal slack variable
u = m.addVar(lb=0.0, obj=1, vtype=GRB.INTEGER, name="u")

#### Step 6: Set your objective function

In [7]:
m.modelSense = GRB.MINIMIZE

#### Step 7: Add Constraints

In [8]:
# defining space constraint
for k in range(K):
    running_sum = 0
    for i in range(I[k]):
        running_sum += x[i,k]*space[i,k]
    m.addConstr((s[k] - e[k] + running_sum), GRB.EQUAL, l*w)

# defining the constraint that turns on the social distancing violated binary variable
for k in range(K):
    m.addConstr(10**6*b[k], GRB.GREATER_EQUAL, e[k])

# defining the constraint that makes sure that if there are social distancing violations,
# they take up at most 20% of the dining room
for k in range(K):
    m.addConstr(e[k], GRB.LESS_EQUAL, l * w * 0.2)
    
# defining the constraint that makes sure that each party only occupies one table
for k in range(K):
    for i in range(I[k]):
        m.addConstr(x[i,k], GRB.LESS_EQUAL, 1)

# goal constraint of attaining a certain profit
revenue = 0
for k in range(K):
    for i in range(I[k]):
        revenue += marginal_revenue[party_size[i,k]] * party_size[i,k] * x[i,k]
m.addConstr(revenue + u, GRB.EQUAL, REVENUE_GOAL)
    
m.update()

#### Step 8: Solve the model

In [9]:
m.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 160 rows, 160 columns and 450 nonzeros
Model fingerprint: 0x464f0fa2
Variable types: 0 continuous, 160 integer (145 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [1e+00, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+04]
Found heuristic solution: objective 20000.000000
Presolve removed 159 rows and 154 columns
Presolve time: 0.00s
Presolved: 1 rows, 6 columns, 6 nonzeros
Found heuristic solution: objective 9407.0000000
Variable types: 0 continuous, 6 integer (1 binary)

Root relaxation: objective 9.335000e+03, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    9335.0000000 9335.00000  0.00%     -    0s

Explored 0 nodes (1 simplex iterations) in 0.05 seconds
Thread count was 4 (of 4 availabl

#### Step 8: Print variable values  (The Messy Way)

In [10]:
print("Printing result\n")
print('Deviation from goals: %g' % m.objVal) #gets the objective function value
print('\nSOLUTION')
for myVars in m.getVars():
    print('%s %g' % (myVars.varName, myVars.x))

Printing result

Deviation from goals: 9335

SOLUTION
x_00 1
x_10 1
x_20 1
x_30 1
x_40 1
x_50 1
x_60 1
x_70 1
x_80 1
x_90 1
x_01 1
x_11 1
x_21 1
x_31 1
x_41 1
x_51 1
x_61 1
x_71 1
x_81 1
x_91 1
x_101 1
x_111 1
x_121 1
x_131 1
x_141 1
x_151 1
x_161 1
x_02 1
x_12 1
x_22 1
x_32 0
x_42 1
x_52 0
x_62 0
x_72 1
x_82 0
x_92 1
x_102 0
x_112 1
x_122 1
x_132 1
x_142 1
x_152 1
x_162 1
x_172 1
x_182 1
x_192 1
x_202 1
x_212 1
x_222 1
x_03 1
x_13 1
x_23 1
x_33 1
x_43 1
x_53 1
x_63 1
x_73 1
x_83 1
x_93 1
x_103 1
x_113 1
x_123 1
x_133 1
x_143 1
x_04 1
x_14 1
x_24 1
x_34 1
x_44 1
x_54 0
x_64 0
x_74 1
x_84 1
x_94 1
x_104 0
x_114 1
x_124 0
x_134 1
x_144 1
x_154 1
x_164 0
x_174 0
x_184 1
x_194 1
x_204 0
x_214 1
x_224 0
x_234 0
x_244 0
x_254 1
x_05 0
x_15 1
x_25 -0
x_35 1
x_45 0
x_55 0
x_65 1
x_75 0
x_85 0
x_95 1
x_105 1
x_115 0
x_125 0
x_135 -0
x_145 1
x_155 1
x_165 0
x_175 1
x_185 0
x_195 1
x_205 1
x_215 1
x_225 1
x_235 0
x_245 1
x_255 0
x_265 0
x_275 1
x_285 -0
x_295 0
x_305 1
x_315 0
x_325 0
x_335 0
x_3

In [11]:
print('Model Summary\n\nObjective Function Value: %g' % m.objVal) #gets the objective function value

if u.x == 0:
    print(f"Revenue goal of ${REVENUE_GOAL} for the day achieved!\n")
else:
    print(f"Missed revenue goal by ${u.x}")
    
for k in range(K):
    res = ""
    if b[k].x != 1:
        res = "not"
    
    print(f"\nTimeslot {k+1}:\n")
    sum = 0
    for i in range(I[k]):
        if(x[i,k].x==1):
            sum += 1
            print(f"party {i+1} of size {party_size[i,k]} was seated")
        else:
            print(f"party {i+1} of size {party_size[i,k]} was not seated")

    print(f"\nPhysical distancing guideline {res} broken")
    print(f"Total number of parties seated during timeslot {k+1}: \t{sum} = {sum/I[k]*100:.2f}%")
    if b[k].x == 1:
        print(f"Amount of overoccupied space: {e[k].x}m^2\n")
    else:
        print(f"Amount of unoccupied space that can be allocated toward more customers: {s[k].x}m^2\n")

Model Summary

Objective Function Value: 9335
Missed revenue goal by $9335.0

Timeslot 1:

party 1 of size 5 was seated
party 2 of size 6 was seated
party 3 of size 3 was seated
party 4 of size 6 was seated
party 5 of size 2 was seated
party 6 of size 5 was seated
party 7 of size 4 was seated
party 8 of size 2 was seated
party 9 of size 5 was seated
party 10 of size 2 was seated

Physical distancing guideline not broken
Total number of parties seated during timeslot 1: 	10 = 100.00%
Amount of unoccupied space that can be allocated toward more customers: 99.0m^2


Timeslot 2:

party 1 of size 2 was seated
party 2 of size 2 was seated
party 3 of size 2 was seated
party 4 of size 3 was seated
party 5 of size 2 was seated
party 6 of size 4 was seated
party 7 of size 4 was seated
party 8 of size 3 was seated
party 9 of size 2 was seated
party 10 of size 2 was seated
party 11 of size 5 was seated
party 12 of size 2 was seated
party 13 of size 3 was seated
party 14 of size 4 was seated
party 